In [1]:
import numpy as np
import pinocchio as pin
from example_robot_data import load
robot = load("go2")

import sys
import os
# Example: add ./local_modules before everything else
local_path = os.path.abspath("../")
sys.path.insert(0, local_path)



from go2meshcat.meshcat_viewer_wrapper import MeshcatVisualizer # the meshcat visualiser
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [2]:
from enum import IntEnum
    
EffectorId = IntEnum('EffectorId', [('RR_FOOT', 0), ('RL_FOOT', 1), ('FL_FOOT', 2),('FR_FOOT',3)])  
EFFECTOR_NAMES = ['RR_foot_joint', 'RL_foot_joint', 'FL_foot_joint', 'FR_foot_joint']
ROOT_NAME = 'root_joint'

In [30]:
from pink.tasks import FrameTask, PostureTask
import pink

def position_task_3d(pos):   
    """
    creates a pink effector task 
    """
    name = EFFECTOR_NAMES[pos[0]]
    task = FrameTask(name, 
                     position_cost=1.0,              # [cost] / [m]
                     orientation_cost=0.001)           # [cost] / [rad])
    target = pin.SE3(np.eye(3), pos[1])
    task.set_target(target)
    return task
                                      
def posture_task(q):
    """
    creates a posture task
    """
    task = PostureTask(cost=1e-6)
    task.set_target(q)
    return task

def root_orientation_task():
    name = ROOT_NAME
    task = FrameTask(name, 
                     position_cost=0.,              # [cost] / [m]
                     orientation_cost=0.1)           # [cost] / [rad])
    target = pin.SE3(np.eye(3), np.zeros(3))
    task.set_target(target)
    return task
    

def to_posture_3d(robot, pos_list, q0 = robot.q0):    
    """
    returns q such that all positions in pos_list are reached with the effector. 
    pos_list is a list of pairs (EffectorId, position). places the root at an average position, then calls ik to place
    effectors at position
    """
    configuration = pink.Configuration(robot.model, robot.data, q0)
    tasks = [posture_task(robot.q0)] + [position_task_3d(pos) for pos in pos_list] + [root_orientation_task()]
    print(len(tasks))
    dt = 6e-3  # [s]
    for t in np.arange(0.0, 100.0, dt):
        velocity = pink.solve_ik(configuration, tasks, dt, solver="quadprog")
        configuration.integrate_inplace(velocity, dt)
    return configuration.q

In [17]:
#test plan

from test_sl1m import plan



#testing this
from numpy import array

targets = [array([1.2, 0.33518737, 0.2]), 
           array([1.2, 0.39140121, 0.2]),
           array([1.23614836, 0.6, 0.2]),
           array([1.22513471, 0.25653363, 0.2]) ]

targets = [array([-0.03200181,  0.29134852,  0.        ]),
 array([0.3       , 0.41161446, 0.05      ]),
 array([0.31744726, 0.57358224, 0.05      ]),
 array([0.3       , 0.27404433, 0.1      ])]

COSTS = {"step_size": [1.0]}

targets = plan()
import time

ProblemData: 
 	 Success: True
 	 Time: 127.57819399121217
Optimized number of steps:               24
Total time is:                           224.2988779908046
Computing the surfaces takes             0.025152985472232103
Computing the initial contacts takes     0.04184001591056585
Generating the problem dictionary takes  54.45624201092869
Solving the problem takes                169.77564297849312
The LP and QP optimizations take         127.57819399121217


In [ ]:

plan_length = len(targets[0])

q = robot.q0.copy()
qs = []

for i in range(plan_length):
    current_targets = [el[i] for el in targets]
    q = to_posture_3d(robot, [el for el in zip(list(EffectorId),current_targets ) ], q)
    qs += [q.copy()]
    viz.display(q)
    #display effectors positions just to make sure that there is no offset
    effectorspheres_id = ['world/ball'+eff for eff in EFFECTOR_NAMES]
    [viz.addSphere(sph_id,.023,[1,0,0,1]) for sph_id in effectorspheres_id]
    
    for sph_id, target in zip(effectorspheres_id, current_targets):
        viz.applyConfiguration(sph_id,[target[0],target[1],target[2],1,0,0,0])
    time.sleep(0.05)

6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6


In [6]:
targets


[[array([-0.46420284, -0.06349486,  0.        ]),
  array([-0.46420284, -0.06349486,  0.        ]),
  array([-0.16589916,  0.17128584,  0.        ]),
  array([-0.16589916,  0.17128584,  0.        ]),
  array([-0.16589916,  0.17128584,  0.        ]),
  array([-0.16589916,  0.17128584,  0.        ]),
  array([-0.01695149,  0.16870595,  0.        ]),
  array([-0.01695149,  0.16870595,  0.        ]),
  array([-0.01695149,  0.16870595,  0.        ]),
  array([-0.01695149,  0.16870595,  0.        ]),
  array([0.3       , 0.20945147, 0.1       ]),
  array([0.3       , 0.20945147, 0.1       ]),
  array([0.3       , 0.20945147, 0.1       ]),
  array([0.3       , 0.20945147, 0.1       ]),
  array([0.6       , 0.24746598, 0.2       ]),
  array([0.6       , 0.24746598, 0.2       ]),
  array([0.6       , 0.24746598, 0.2       ]),
  array([0.6       , 0.24746598, 0.2       ]),
  array([0.9       , 0.28585135, 0.3       ]),
  array([0.9       , 0.28585135, 0.3       ]),
  array([0.9       , 0.2858513